In [1]:
import pandas as pd
import geopandas as gpd

# Dicionário com siglas e nomes completos dos estados brasileiros.
estados = {
    "AC": "Acre", "AL": "Alagoas", "AP": "Amapá", "AM": "Amazonas",
    "BA": "Bahia", "CE": "Ceará", "DF": "Distrito Federal", "ES": "Espírito Santo",
    "GO": "Goiás", "MA": "Maranhão", "MT": "Mato Grosso", "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais", "PA": "Pará", "PB": "Paraíba", "PR": "Paraná",
    "PE": "Pernambuco", "PI": "Piauí", "RJ": "Rio de Janeiro", "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul", "RO": "Rondônia", "RR": "Roraima", "SC": "Santa Catarina",
    "SP": "São Paulo", "SE": "Sergipe", "TO": "Tocantins"
}

def main():
    # Solicita ao usuário o estado e a cidade desejada
    sigla_estado = input("Informe a sigla do estado: ").strip().upper()
    if sigla_estado not in estados:
        print("Sigla de estado inválida. Verifique e tente novamente.")
        return
    nome_estado = estados[sigla_estado]

    cidade = input("Informe o nome da cidade: ").strip()

    # Leitura do arquivo de UFs do Brasil
    uf_path = "brasil_UFs_simp.geojson"
    try:
        br = gpd.read_file(uf_path)
    except Exception as e:
        print(f"Erro ao ler o arquivo de UFs: {e}")
        return

    # Filtra removendo o estado que terá o detalhe de municípios
    br = br[br["name"] != nome_estado]

    # Adiciona o prefixo "PRO-" na coluna id
    br["id"] = "PRO-" + br["id"].astype(str)

    # Construção do caminho para o arquivo de municípios do estado escolhido
    cidades_path = f"SIMP/{sigla_estado}_Municipios.geojson"
    try:
        cidades = gpd.read_file(cidades_path)
    except Exception as e:
        print(f"Erro ao ler o arquivo de municípios: {e}")
        return

    # Separa a cidade de interesse dos demais municípios
    cidade_sel = cidades[cidades["name"].str.strip().str.lower() == cidade.lower()]
    if cidade_sel.empty:
        print("Cidade não encontrada no arquivo de municípios.")
        return
    # Remove a cidade selecionada dos demais municípios
    cidades = cidades[cidades["name"].str.strip().str.lower() != cidade.lower()]

    # Atualiza os IDs dos municípios
    cidades["id"] = "MUN-" + cidades["id"].astype(str)

    # Atualiza o id da cidade selecionada e adiciona a coluna poly_type
    cidade_sel["id"] = "AOI-" + cidade_sel["id"].astype(str)
    cidade_sel["poly_type"] = "core"

    # Concatena a camada de UFs, os municípios do estado e a cidade escolhida
    projeto = pd.concat([br, cidades, cidade_sel], ignore_index=True)

    # Salva o projeto final para um arquivo geojson com nome baseado na cidade selecionada
    output_filename = f"{cidade}_AOI.geojson"
    try:
        projeto.to_file(output_filename, driver='GeoJSON')
        print(f"Projeto salvo com sucesso em {output_filename}")
    except Exception as e:
        print(f"Erro ao salvar o projeto: {e}")

if __name__ == "__main__":
    main()


Informe a sigla do estado:  SP
Informe o nome da cidade:  São Bernardo do Campo


Projeto salvo com sucesso em São Bernardo do Campo_AOI.geojson


In [2]:
teste = gpd.read_file("São Bernardo do Campo_AOI.geojson")

In [4]:
teste

,id,name,poly_type,geometry
0,PRO-12,Acre,periphery,"POLYGON ((-73.80156 -7.11182, -73.8014 -7.1126..."
1,PRO-13,Amazonas,periphery,"POLYGON ((-58.13706 -7.35612, -58.14737 -7.343..."
2,PRO-15,Pará,periphery,"MULTIPOLYGON (((-46.10411 -1.20197, -46.09817 ..."
3,PRO-16,Amapá,periphery,"MULTIPOLYGON (((-54.87225 2.43375, -54.87625 2..."
4,PRO-17,Tocantins,periphery,"POLYGON ((-50.51098 -12.8609, -50.50742 -12.85..."
...,...,...,...,...
666,MUN-3557105,Votuporanga,periphery,"POLYGON ((-50.12017 -20.52165, -50.11761 -20.5..."
667,MUN-3557154,Zacarias,periphery,"POLYGON ((-49.96007 -21.13312, -49.99693 -21.0..."
668,MUN-3557204,Chavantes,periphery,"POLYGON ((-49.81939 -23.07695, -49.81577 -23.0..."
669,MUN-3557303,Estiva Gerbi,periphery,"POLYGON ((-46.89708 -22.20488, -46.89844 -22.2..."


In [4]:
teste

,id,name,poly_type,geometry
0,PRO-12,Acre,periphery,"POLYGON ((-73.80156 -7.11182, -73.8014 -7.1126..."
1,PRO-13,Amazonas,periphery,"POLYGON ((-58.13706 -7.35612, -58.14737 -7.343..."
2,PRO-15,Pará,periphery,"MULTIPOLYGON (((-46.10411 -1.20197, -46.09817 ..."
3,PRO-16,Amapá,periphery,"MULTIPOLYGON (((-54.87225 2.43375, -54.87625 2..."
4,PRO-17,Tocantins,periphery,"POLYGON ((-50.51098 -12.8609, -50.50742 -12.85..."
...,...,...,...,...
113,MUN-3306107,Valença,periphery,"POLYGON ((-43.66845 -22.08613, -43.669 -22.087..."
114,MUN-3306156,Varre-Sai,periphery,"POLYGON ((-41.91182 -20.92891, -41.9114 -20.92..."
115,MUN-3306206,Vassouras,periphery,"POLYGON ((-43.68843 -22.46305, -43.68823 -22.4..."
116,MUN-3306305,Volta Redonda,periphery,"POLYGON ((-44.00564 -22.40709, -44.01314 -22.4..."
